In [1]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
import sklearn
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#%matplotlib inline

In [2]:
#Preprocesing the training set
train_data_generator = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2, horizontal_flip=True, validation_split=0.2)
train_data_set = train_data_generator.flow_from_directory('E:/Data Science/Hackathon/train_images',target_size=(224,224),
                                                          subset='training',batch_size=32,class_mode='binary')

Found 3445 images belonging to 2 classes.


In [3]:
train_data_set.filenames[0]

'0\\00006497_004.png'

In [4]:
#Preprocesing the training set
val_data_set = train_data_generator.flow_from_directory('E:/Data Science/Hackathon/train_images',target_size=(224,224),
                                                        subset = 'validation',batch_size=32,class_mode='binary')

Found 860 images belonging to 2 classes.


In [5]:
#DEveloping model 2
model3 = tf.keras.models.Sequential()
model3.add(tf.keras.layers.Conv2D(128,(3,3),activation='relu'))
model3.add(tf.keras.layers.MaxPool2D())
#model2.add(tf.keras.layers.Conv2D(32,(3,3),activation='relu'))
#model2.add(tf.keras.layers.MaxPool2D())
#model2.add(tf.keras.layers.Conv2D(32,(3,3),activation='relu',padding='same'))
#model2.add(tf.keras.layers.MaxPool2D())
model3.add(tf.keras.layers.Flatten())
model3.add(tf.keras.layers.Dense(128,activation='relu',kernel_initializer='he_uniform'))
model3.add(tf.keras.layers.Dropout(0.4))
model3.add(tf.keras.layers.Dense(64,activation='relu',kernel_initializer='he_uniform'))
model3.add(tf.keras.layers.Dropout(0.4))
model3.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

In [8]:
#compile the model
model3.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
#Training the model
history1=model3.fit(x=train_data_set,validation_data=val_data_set, epochs=20,batch_size=32,verbose=1)

Epoch 1/20
108/108 [==============================] - 346s 3s/step - loss: 0.7899 - accuracy: 0.6186 - val_loss: 0.6551 - val_accuracy: 0.6430
Epoch 2/20
108/108 [==============================] - 202s 2s/step - loss: 0.6437 - accuracy: 0.6418 - val_loss: 0.6320 - val_accuracy: 0.6430
Epoch 3/20
108/108 [==============================] - 202s 2s/step - loss: 0.6301 - accuracy: 0.6441 - val_loss: 0.6415 - val_accuracy: 0.6430
Epoch 4/20
108/108 [==============================] - 197s 2s/step - loss: 0.6254 - accuracy: 0.6438 - val_loss: 0.6317 - val_accuracy: 0.6430
Epoch 5/20
108/108 [==============================] - 197s 2s/step - loss: 0.6203 - accuracy: 0.6496 - val_loss: 0.6165 - val_accuracy: 0.6465
Epoch 6/20
108/108 [==============================] - 151s 1s/step - loss: 0.6198 - accuracy: 0.6517 - val_loss: 0.6375 - val_accuracy: 0.6663
Epoch 7/20
108/108 [==============================] - 85s 785ms/step - loss: 0.6120 - accuracy: 0.6589 - val_loss: 0.6289 - val_accuracy: 0.66

In [11]:
model3.save('C:/Users/Mrinal Kalita/Python Projects/Hackathon/model3.h5')

In [13]:
f = 'E:/Data Science/Hackathon/test_images'

In [14]:
import cv2
import os

def load_images_from_folder(folder):
    images = []
    file_name = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.resize(img, (64, 64))
        img = img/255
        if img is not None:
            images.append(img)
            file_name.append(filename)
    return images, file_name

In [15]:
test_img, file = load_images_from_folder(f)

In [16]:
test_img = np.array(test_img).reshape(-1,64,64,3)

In [18]:
def sub(model,test,filename):
    pred = model.predict_classes(test)
    pred = pred.flatten()
    dic = {'Image Index':filename,'Finding Labels':pred}
    sub = pd.DataFrame(dic)
    return sub.to_csv('Sample submission.csv',index=False)

In [19]:
sub(model3,test_img,file)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


# Model2

In [6]:
#loading data set
df = pd.read_csv("E:/Data Science/Hackathon/train_set.csv")

In [7]:
df = df.drop('Patient ID',axis=1)

In [8]:
df['Finding Labels'] = df['Finding Labels'].astype('category')

In [9]:
df = df.drop(['OriginalImagePixelSpacing_x','OriginalImagePixelSpacing_y'],axis=1)

In [10]:
# Computing 10th, 90th percentiles and replacing the outlier treatment in python
tenth_percentile = np.percentile(df['Follow-up'], 10) 
ninetieth_percentile = np.percentile(df['Follow-up'], 90) 
# print(tenth_percentile, ninetieth_percentile)b = 
b= np.where(df['Follow-up']<tenth_percentile, tenth_percentile, df['Follow-up']) 
b = np.where(df['Follow-up']>ninetieth_percentile, ninetieth_percentile, df['Follow-up']) 
# print("Sample:", sample) 
df['Follow-up'] = b

In [11]:
# Computing 10th, 90th percentiles and replacing the outlier treatment in python
tenth_percentile = np.percentile(df['OriginalImage_Width'], 10) 
ninetieth_percentile = np.percentile(df['OriginalImage_Width'], 90) 
# print(tenth_percentile, ninetieth_percentile)b = 
d = np.where(df['OriginalImage_Width']<tenth_percentile, tenth_percentile, df['OriginalImage_Width']) 
d = np.where(df['OriginalImage_Width']>ninetieth_percentile, ninetieth_percentile, df['OriginalImage_Width']) 
# print("Sample:", sample) 
df['OriginalImage_Width'] = d

In [12]:
df.head()

,Image Index,Finding Labels,Follow-up,Patient Age,Patient Gender,View Position,OriginalImage_Width,OriginalImage_Height
0,00000011_000.png,0,0.0,75,M,PA,2638.0,2449
1,00000013_011.png,1,11.0,60,M,AP,3056.0,2544
2,00000013_012.png,1,12.0,60,M,AP,3056.0,2544
3,00000013_013.png,1,13.0,60,M,AP,3056.0,2544
4,00000013_036.png,1,32.0,56,M,PA,2952.0,2968


In [13]:
img_ind = []
label=[]
follow = []
age = []
gen = []
pos = [] 
wid = []
hei =[]
for i in train_data_set.filenames:
    ind = df[df['Image Index']==i[2:]].index.values[0]
    img_ind.append(df['Image Index'][ind])
    label.append(df['Finding Labels'][ind])
    follow.append(df['Follow-up'][ind])
    age.append(df['Patient Age'][ind])
    gen.append(df['Patient Gender'][ind])
    pos.append(df['View Position'][ind])
    wid.append(df['OriginalImage_Width'][ind])
    hei.append(df['OriginalImage_Height'][ind])

In [14]:
dic = {'Image Index':img_ind,'Finding Labels':label,'Follow-up':follow,'Patient Age':age,'Patient Gender':gen,'View Position':pos,
       'OriginalImage_Width':wid,'OriginalImage_Height':hei}
df_train = pd.DataFrame(dic)

In [15]:
img_ind = []
label=[]
follow = []
age = []
gen = []
pos = [] 
wid = []
hei =[]
for i in val_data_set.filenames:
    ind = df[df['Image Index']==i[2:]].index.values[0]
    img_ind.append(df['Image Index'][ind])
    label.append(df['Finding Labels'][ind])
    follow.append(df['Follow-up'][ind])
    age.append(df['Patient Age'][ind])
    gen.append(df['Patient Gender'][ind])
    pos.append(df['View Position'][ind])
    wid.append(df['OriginalImage_Width'][ind])
    hei.append(df['OriginalImage_Height'][ind])

In [16]:
dic = {'Image Index':img_ind,'Finding Labels':label,'Follow-up':follow,'Patient Age':age,'Patient Gender':gen,'View Position':pos,
       'OriginalImage_Width':wid,'OriginalImage_Height':hei}
df_val = pd.DataFrame(dic)

In [17]:
df_train_f = df_train.drop('Image Index', axis=1)
df_val_f = df_val.drop('Image Index', axis =1)

In [18]:
#Label encoding
d = {'M':0,'F':1,'PA':0,'AP':1}
df_train_f= df_train_f.replace(d)
df_val_f= df_val_f.replace(d)

In [20]:
x_train = np.array(df_train_f.drop('Finding Labels',axis=1))
y_train = df_train_f['Finding Labels']
x_val = np.array(df_val_f.drop('Finding Labels',axis=1))
y_val= df_val_f['Finding Labels']

In [21]:
from sklearn.preprocessing import StandardScaler

In [22]:
#Scaling
sc= StandardScaler()
x_train = sc.fit_transform(x_train)
x_val = sc.transform(x_val)

In [23]:
x_train.shape[1]

6

In [24]:
extra_nb_features = x_train.shape[1]

inp = tf.keras.Input(shape=(64,1))
extra_inp = tf.keras.Input(shape=(extra_nb_features, ))

#x = Embedding(embedding_matrix.shape[0], 300, weights=[embedding_matrix], trainable=False)(inp)
#x = SpatialDropout1D(0.4)(x)

x = tf.keras.layers.Conv1D(32, 3, activation='relu')(inp)
x = tf.keras.layers.MaxPooling1D()(x)

x = tf.keras.layers.Dropout(0.2)(x)  
x = tf.keras.layers.Flatten()(x)
combined = tf.keras.layers.Concatenate(axis=-1)([x, extra_inp])

combined = tf.keras.layers.Dropout(0.2)(combined)

outp = tf.keras.layers.Dense(128, activation="relu")(combined)
outp = tf.keras.layers.Dropout(0.2)(outp)
outp = tf.keras.layers.Dense(64, activation="relu")(outp)
outp = tf.keras.layers.Dense(1, activation="sigmoid")(outp)

model = tf.keras.Model(inputs=[inp, extra_inp] , outputs=outp)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 1)]      0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 62, 32)       128         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling1d (MaxPooling1D)    (None, 31, 32)       0           conv1d[0][0]                     
__________________________________________________________________________________________________
dropout_2 (Dropout)             (None, 31, 32)       0           max_pooling1d[0][0]              
_______________________________________________________________________________________

In [25]:
#compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [31]:
model.fit(x= [train_data_set, x_train],y = y_train,epochs=20,batch_size=32)

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'numpy.ndarray'>", "<class 'tensorflow.python.keras.preprocessing.image.DirectoryIterator'>"}), <class 'pandas.core.series.Series'>

In [128]:
from tensorflow.keras.layers import Input, Conv1D, Flatten, Concatenate, Dense
from tensorflow.keras import Model

inputs = Input(shape= (32,1))
h = Conv1D(filters= 16, kernel_size= 5, activation= 'relu', padding= 'same')(inputs)
h = Flatten()(h)
aux_inputs = Input(shape= (12,))
h = Concatenate()([h, aux_inputs])
output = Dense(20, activation= 'sigmoid')(h)

model = Model(inputs= [inputs, aux_inputs], outputs= output)

In [129]:
#compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [130]:
model.fit([train_data_set, x_train],epochs=20,batch_size=32)

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'tensorflow.python.keras.preprocessing.image.DirectoryIterator'>", "<class 'numpy.ndarray'>"}), <class 'NoneType'>

In [133]:
#DEveloping model 3
model4 = tf.keras.models.Sequential()
model4.add(tf.keras.layers.Conv2D(32,(3,3),activation='relu'))
model4.add(tf.keras.layers.MaxPool2D())
model4.add(tf.keras.layers.Conv2D(32,(3,3),activation='relu'))
model4.add(tf.keras.layers.MaxPool2D())
model4.add(tf.keras.layers.Flatten())
model4.add(tf.keras.layers.Dense(128,activation='relu',kernel_initializer='he_uniform'))
model4.add(tf.keras.layers.Dropout(0.4))
model4.add(tf.keras.layers.Dense(64,activation='relu',kernel_initializer='he_uniform'))
model4.add(tf.keras.layers.Dropout(0.4))
model4.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

In [134]:
#compile the model
model4.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [135]:
model4.fit(train_data_set,validation_data=val_data_set,epochs=20,batch_size=32)

Epoch 1/20
108/108 [==============================] - 130s 1s/step - loss: 0.6669 - accuracy: 0.6302 - val_loss: 0.6437 - val_accuracy: 0.6430
Epoch 2/20
108/108 [==============================] - 74s 682ms/step - loss: 0.6484 - accuracy: 0.6424 - val_loss: 0.6286 - val_accuracy: 0.6430
Epoch 3/20
108/108 [==============================] - 73s 676ms/step - loss: 0.6311 - accuracy: 0.6511 - val_loss: 0.6275 - val_accuracy: 0.6488
Epoch 4/20
108/108 [==============================] - 82s 761ms/step - loss: 0.6115 - accuracy: 0.6717 - val_loss: 0.6115 - val_accuracy: 0.6674
Epoch 5/20
108/108 [==============================] - 82s 760ms/step - loss: 0.6116 - accuracy: 0.6746 - val_loss: 0.6161 - val_accuracy: 0.6744
Epoch 6/20
108/108 [==============================] - 75s 696ms/step - loss: 0.6062 - accuracy: 0.6729 - val_loss: 0.6167 - val_accuracy: 0.6744
Epoch 7/20
108/108 [==============================] - 77s 711ms/step - loss: 0.6039 - accuracy: 0.6827 - val_loss: 0.6238 - val_accu

In [136]:
model4.save('C:/Users/Mrinal Kalita/Python Projects/Hackathon/model4.h5')

In [10]:
f = 'E:/Data Science/Hackathon/test_images'

In [9]:
import cv2
import os

def load_images_from_folder(folder):
    images = []
    file_name = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.resize(img, (64, 64))
        img = img/255
        if img is not None:
            images.append(img)
            file_name.append(filename)
    return images, file_name

In [11]:
test_img, file = load_images_from_folder(f)

In [12]:
test_img = np.array(test_img).reshape(-1,64,64,3)

In [13]:
def sub(model,test,filename):
    pred = model.predict_classes(test)
    pred = pred.flatten()
    dic = {'Image Index':filename,'Finding Labels':pred}
    sub = pd.DataFrame(dic)
    return sub.to_csv('Sample submission.csv',index=False)

In [142]:
sub(model4,test_img,file)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [5]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

base_model = MobileNet(weights='imagenet',
                       include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x = base_model.output
x = GlobalAveragePooling2D()(x) # Optional
x = Dense(256,activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(64,activation='relu')(x)#we add dense layers so that the model can learn more complex functions and classify for better results.
preds = Dense(1,activation='sigmoid')(x) #final layer with softmax activation

model_mobilenet = Model(inputs=base_model.input,
                        outputs=preds)

# set the first 20 layers of the network to be non-trainable
#for layer in model_mobilenet.layers[:10]:
#    layer.trainable=False
for layer in model_mobilenet.layers:
    layer.trainable=True

# Compile the model
model_mobilenet.compile(optimizer='Adam',
                      loss='bnary_crossentropy',
                      metrics=['accuracy'])

model_mobilenet.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)   

In [6]:
model_mobilenet.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint("model-{val_loss:.2f}.h5", monitor="val_loss", verbose=1,mode='min', save_best_only=True, save_weights_only=True)

stop = EarlyStopping(monitor="val_loss", patience=7, mode='min')

reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=5, min_lr=1e-6, verbose=1, mode='min')

In [8]:
history1 = model_mobilenet.fit(train_data_set,
                                  batch_size=32,
                                  epochs=100,
                                  validation_data=val_data_set,verbose=1, callbacks=[checkpoint,stop, reduce_lr])

Epoch 1/100
108/108 [==============================] - ETA: 0s - loss: 4.2528e-08 - accuracy: 0.5306
Epoch 00001: val_loss improved from inf to 0.00000, saving model to model-0.00.h5
108/108 [==============================] - 101s 932ms/step - loss: 4.2528e-08 - accuracy: 0.5306 - val_loss: 4.2555e-08 - val_accuracy: 0.5267
Epoch 2/100
108/108 [==============================] - ETA: 0s - loss: 4.2528e-08 - accuracy: 0.5231
Epoch 00002: val_loss did not improve from 0.00000
108/108 [==============================] - 94s 869ms/step - loss: 4.2528e-08 - accuracy: 0.5231 - val_loss: 4.2555e-08 - val_accuracy: 0.5349
Epoch 3/100
108/108 [==============================] - ETA: 0s - loss: 4.2528e-08 - accuracy: 0.5254
Epoch 00003: val_loss did not improve from 0.00000
108/108 [==============================] - 90s 834ms/step - loss: 4.2528e-08 - accuracy: 0.5254 - val_loss: 4.2555e-08 - val_accuracy: 0.5151
Epoch 4/100
108/108 [==============================] - ETA: 0s - loss: 4.2528e-08 - ac

In [14]:
sub(model_mobilenet,test_img,file)

AttributeError: 'Functional' object has no attribute 'predict_classes'

In [29]:
s = model_mobilenet.predict(test_img) > 0.5
s[0].astype(np.int)

array([0])

In [30]:
def sub(model,test,filename):
    pred = model.predict(test)
    pred = pred >0.5
    pred= pred.astype(np.int)
    pred = pred.flatten()
    dic = {'Image Index':filename,'Finding Labels':pred}
    sub = pd.DataFrame(dic)
    return sub.to_csv('Sample submission.csv',index=False)

In [31]:
sub(model_mobilenet,test_img,file)

In [32]:
from tensorflow.keras import backend
backend.clear_session()
#Fixing the seed for random number generators so that we can ensure we receive the same output everytime
np.random.seed(47)
import random
random.seed(47)
tf.random.set_seed(47)

In [54]:
from tensorflow.keras.applications import MobileNet, efficientnet,EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, MaxPool2D
from tensorflow.keras.models import Model

base_model = EfficientNetB7(weights='imagenet',
                       include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x = base_model.output
x = GlobalAveragePooling2DD()(x) # Optional
x = Dense(256,activation='relu')(x)
x = Dropout(0.4)(x)
x = Dense(64,activation='relu')(x)#we add dense layers so that the model can learn more complex functions and classify for better results.
preds = Dense(1,activation='sigmoid')(x) #final layer with softmax activation

model_enet = Model(inputs=base_model.input,
                        outputs=preds)

# set the first 20 layers of the network to be non-trainable
for layer in model_enet.layers:
    layer.trainable=False
#for layer in model_enet.layers[800:]:
 #   layer.trainable=True

# Compile the model
model_enet.compile(optimizer='Adam',
                      loss='bnary_crossentropy',
                      metrics=['accuracy'])

model_enet.summary()

ResourceExhaustedError: OOM when allocating tensor with shape[1,1,288,48] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Add]

In [ ]:
model_enet.compile(optimizer=tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint("model-{val_loss:.2f}.h5", monitor="val_loss", verbose=1,mode='min', save_best_only=True, save_weights_only=True)

stop = EarlyStopping(monitor="val_loss", patience=9, mode='min')

reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=5, min_lr=1e-6, verbose=1, mode='min')

In [ ]:
history2 = model_enet.fit(train_data_set,
                                  batch_size=32,
                                  epochs=100,
                                  validation_data=val_data_set,verbose=1, callbacks=[checkpoint,stop, reduce_lr])

In [ ]:
from tensorflow.keras.applications import MobileNet, efficientnet,EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, MaxPool2D
from tensorflow.keras.models import Model
model = EfficientNetB0(
    include_top=True,
    weights=None,
    classes=1,
    input_shape=(64, 64, 3),
)
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()

epochs = 40  # @param {type: "slider", min:10, max:100}
hist = model.fit(train_data_set, epochs=40, validation_data=val_data_set)

Model: "efficientnetb0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
rescaling_2 (Rescaling)         (None, 64, 64, 3)    0           input_3[0][0]                    
__________________________________________________________________________________________________
normalization_2 (Normalization) (None, 64, 64, 3)    7           rescaling_2[0][0]                
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 65, 65, 3)    0           normalization_2[0][0]            
_____________________________________________________________________________________

In [8]:
from tensorflow.keras.applications import MobileNet, efficientnet,EfficientNetB0,DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, MaxPool2D
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
base_model = tf.keras.applications.DenseNet121(input_shape=(224,224,3),
                                           include_top=False,
                                           weights='imagenet',
                                           pooling="avg")

base_model.trainable = False

x = base_model.output

x = Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
x = Dropout(0.2)(x)
x = Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
x = Dropout(0.2)(x)
x = Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
x = Dropout(0.2)(x)
x = Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
x = Dropout(0.2)(x)

predictions = Dense(1, activation='sigmoid',name='Final')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(loss = 'binary_crossentropy',    
              optimizer="Adam",
              metrics=["Accuracy"]
                             )
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_4 (ZeroPadding2D (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_4[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
_______________________________________________________________________________________

In [10]:
history = model.fit(
                    train_data_set, 
                    validation_data= val_data_set,     
                    epochs=20)
                    #use_multiprocessing=True,
            
                    #callbacks=[reduced_lr, cp_callback, auroc]

Epoch 1/20
108/108 [==============================] - 357s 3s/step - loss: 0.8438 - accuracy: 0.0000e+00 - val_loss: 0.7979 - val_accuracy: 0.0000e+00
Epoch 2/20
 54/108 [==============>...............] - ETA: 1:41 - loss: 0.7767 - accuracy: 0.0000e+00

KeyboardInterrupt: 